<a href="https://colab.research.google.com/github/AUT-Student/CN-Project/blob/main/CN_Project_Q2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Libraries

In [1]:
import torch
import scipy
import scipy.sparse as sp
from torch.nn import Linear, Softmax
import numpy as np
import torch.nn.functional as F

In [2]:
!pip install pyg-lib torch-scatter torch-sparse -f https://data.pyg.org/whl/torch-1.13.0+cu116.html
!pip install torch-geometric
!pip install torch-cluster torch-spline-conv -f https://data.pyg.org/whl/torch-1.13.0+cu116.html

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://data.pyg.org/whl/torch-1.13.0+cu116.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 22.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 64.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 59.3 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 565.0/565.0 KB 7.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.2/280.2 KB 20.2 MB/s eta 0:00:00
  Created wheel for torch-geometric: filename=torch_geometric-2.2.0-py3-none-any.whl size=773302 sha256=4713926b8264d237fe034f9432ff97f0d3055c8b21429c3aa62f54022185a5fc
  Stored in directory: /root/.cache/pip/wheels/59/a3/20/198928106d3169865ae73afcbd3d3d1796cf6b429b55c65378
Successfully built 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://data.pyg.org/whl/torch-1.13.0+cu116.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 33.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 873.9/873.9 KB 55.6 MB/s eta 0:00:00


In [3]:
from torch_geometric.utils import to_scipy_sparse_matrix

# Datasets

In [311]:
from torch_geometric.datasets.amazon import Amazon
from torch_geometric.datasets.planetoid import Planetoid
from torch_geometric.transforms.random_node_split import RandomNodeSplit

## Amazon Computers

In [88]:
amazon_computers = Amazon("/content", "computers")[0]

Processing...
Done!


In [91]:
amazon_computers

Data(x=[13752, 767], edge_index=[2, 491722], y=[13752])

In [92]:
set(amazon_computers.y.numpy())

{0, 1, 2, 3, 4, 5, 6, 7, 8, 9}

In [312]:
rns = RandomNodeSplit(num_test=4000, num_val=4000)

In [313]:
rns(amazon_computers)

Data(x=[13752, 767], edge_index=[2, 491722], y=[13752], train_mask=[13752], val_mask=[13752], test_mask=[13752])

## Amazon Photo

In [320]:
amazon_photo = Amazon("/content", "photo")[0]

In [321]:
amazon_photo

Data(x=[7650, 745], edge_index=[2, 238162], y=[7650])

In [322]:
set(amazon_photo.y.numpy())

{0, 1, 2, 3, 4, 5, 6, 7}

In [323]:
rns = RandomNodeSplit(num_test=2000, num_val=2000)

In [324]:
rns(amazon_photo)

Data(x=[7650, 745], edge_index=[2, 238162], y=[7650], train_mask=[7650], val_mask=[7650], test_mask=[7650])

## PubMed

In [152]:
pubmed = Planetoid("/content/planetoid", name="PubMed")[0]
print(pubmed)
print(set(pubmed.y.numpy()))

Data(x=[19717, 500], edge_index=[2, 88648], y=[19717], train_mask=[19717], val_mask=[19717], test_mask=[19717])
{0, 1, 2}


In [308]:
sum(pubmed.train_mask)

tensor(60)

In [309]:
sum(pubmed.test_mask)

tensor(1000)

In [310]:
sum(pubmed.val_mask)

tensor(500)

In [153]:
cora = Planetoid("/content/planetoid", name="cora")[0]
print(cora)
print(set(cora.y.numpy()))

Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])
{0, 1, 2, 3, 4, 5, 6}


Processing...
Done!


In [155]:
citeseer = Planetoid("/content/planetoid", name="citeseer")[0]
print(citeseer)
print(set(citeseer.y.numpy()))

Data(x=[3327, 3703], edge_index=[2, 9104], y=[3327], train_mask=[3327], val_mask=[3327], test_mask=[3327])
{0, 1, 2, 3, 4, 5}


Processing...
Done!


# Models

## Model SGC

In [177]:
#Source:
#1.   https://docs.dgl.ai/en/0.8.x/_modules/dgl/nn/pytorch/conv/sgconv.html
#2.   https://github.com/Tiiiger/SGC

class SGC(torch.nn.Module):
    def __init__(self, number_features, number_classes, k):
      super().__init__()
      self.linear = Linear(number_features, number_classes)
      self.softmax = Softmax(dim=1)
      self.k = k
      
      torch.nn.init.xavier_uniform_(self.linear.weight)

    def create_sgc_matrix(self, dataset):
      A = to_scipy_sparse_matrix(dataset.edge_index)
      I = sp.eye(dataset.num_nodes)

      A_tilde = A + I

      D_tilde = sp.diags(A_tilde.sum(axis=0).A1)

      S = D_tilde.power(-1/2) * A * D_tilde.power(-1/2)

      Sk = S.power(self.k)

      X = sp.csr_matrix(dataset.x)

      SkX = Sk * sp.csr_matrix(dataset.x)

      SkX = torch.tensor(SkX.todense()).to(torch.float32)

      return SkX

    def forward(self, x):
      x = self.linear(x)
      return x

## Model GCN

In [334]:
# https://github.com/pyg-team/pytorch_geometric#train-your-own-gnn-model

import torch
from torch import Tensor
from torch_geometric.nn import GCNConv
from torch_geometric.datasets import Planetoid

class GCN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super().__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, out_channels)

    def forward(self, x: Tensor, edge_index: Tensor) -> Tensor:
        # x: Node feature matrix of shape [num_nodes, in_channels]
        # edge_index: Graph connectivity matrix of shape [2, num_edges]
        x = self.conv1(x, edge_index).relu()
        x = self.conv2(x, edge_index)
        return x

# Train and Test

In [ ]:
class EarlyStopper:
    def __init__(self, patience=1, min_delta=0):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.min_validation_loss = np.inf

    def early_stop(self, validation_loss):
        if validation_loss < self.min_validation_loss:
            self.min_validation_loss = validation_loss
            self.counter = 0
        elif validation_loss > (self.min_validation_loss + self.min_delta):
            self.counter += 1
            if self.counter >= self.patience:
                return True
        return False

In [ ]:
def train(model, dataset, lr):
  x = model.create_sgc_matrix(dataset)
  
  model.train()

  optimizer = torch.optim.Adam(model.parameters(), lr=lr,
                           weight_decay=5e-6)
  
  early_stopper = EarlyStopper(patience=100, min_delta=0.001)
  
  for epoch in range(1000):
    pred = model(x)
    train_loss = F.cross_entropy(pred[dataset.train_mask],
                                  dataset.y[dataset.train_mask])
    
    validation_loss = F.cross_entropy(pred[dataset.val_mask],
                                      dataset.y[dataset.val_mask])

    if epoch%50==0:
      print(f"epoch = {epoch} train_loss = {round(float(train_loss), 4)}, validation_loss={round(float(validation_loss), 4)}")

    # Backpropagation
    optimizer.zero_grad()
    train_loss.backward()
    optimizer.step()  

    if early_stopper.early_stop(validation_loss):             
        print(epoch)
        break

In [ ]:
def test(model, dataset):
  x = model.create_sgc_matrix(dataset)
  model.eval()
  with torch.no_grad():
    pred = model(x).argmax(dim=-1)

  accs = []
  for mask in [dataset.train_mask, dataset.val_mask, dataset.test_mask]:
      accs.append(int((pred[mask] == dataset.y[mask]).sum()) / int(mask.sum()))

  print(f"Train Accuracy = {accs[0]}")
  print(f"Valid Accuracy = {accs[1]}")
  print(f"Test  Accuracy = {accs[2]}")  

In [354]:
def train_gcn(model, dataset):
  model.train()

  early_stopper = EarlyStopper(patience=50, min_delta=0.01)
  optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

  for epoch in range(20000):
    pred = model(dataset.x, dataset.edge_index)
    train_loss = F.cross_entropy(pred[dataset.train_mask],
                                  dataset.y[dataset.train_mask])
    
    validation_loss = F.cross_entropy(pred[dataset.val_mask],
                                      dataset.y[dataset.val_mask])

    # Backpropagation
    optimizer.zero_grad()
    train_loss.backward()
    optimizer.step()

    if early_stopper.early_stop(validation_loss):             
        break

In [330]:
def test_gcn(model, dataset):
  model.eval()
  with torch.no_grad():
    pred = model(dataset.x, dataset.edge_index).argmax(dim=-1)

  accs = []
  for mask in [dataset.train_mask, dataset.val_mask, dataset.test_mask]:
      accs.append(int((pred[mask] == dataset.y[mask]).sum()) / int(mask.sum()))

  print(f"Train Accuracy = {accs[0]}")
  print(f"Valid Accuracy = {accs[1]}")
  print(f"Test  Accuracy = {accs[2]}")

# Results

## PubMed

In [280]:
model_pubmed = SGC(number_features=500, number_classes=3, k=2)

In [281]:
train(model_pubmed, pubmed, lr=0.2)

epoch = 0 train_loss = 1.0984, validation_loss=1.102
epoch = 50 train_loss = 0.6733, validation_loss=0.9326
epoch = 100 train_loss = 0.6254, validation_loss=0.9118
epoch = 150 train_loss = 0.6098, validation_loss=0.904
epoch = 200 train_loss = 0.603, validation_loss=0.9
epoch = 250 train_loss = 0.5994, validation_loss=0.8977
epoch = 300 train_loss = 0.5973, validation_loss=0.8962
epoch = 350 train_loss = 0.5959, validation_loss=0.8951
epoch = 400 train_loss = 0.5949, validation_loss=0.8944
epoch = 450 train_loss = 0.5942, validation_loss=0.8939
epoch = 500 train_loss = 0.5937, validation_loss=0.8935
epoch = 550 train_loss = 0.5934, validation_loss=0.8932
epoch = 600 train_loss = 0.5931, validation_loss=0.893
epoch = 650 train_loss = 0.5929, validation_loss=0.8929
epoch = 700 train_loss = 0.5927, validation_loss=0.8927
epoch = 750 train_loss = 0.5926, validation_loss=0.8927
epoch = 800 train_loss = 0.5925, validation_loss=0.8926
epoch = 850 train_loss = 0.5925, validation_loss=0.8925
ep

In [283]:
test(model_pubmed, pubmed)

Train Accuracy = 0.9333333333333333
Valid Accuracy = 0.714
Test  Accuracy = 0.707


## Amazon Computers

In [317]:
model_computers =  SGC(number_features=767, number_classes=10, k=2)

In [318]:
train(model_computers, amazon_computers, lr=0.2)

epoch = 0 train_loss = 2.2926, validation_loss=2.2918
epoch = 50 train_loss = 0.6805, validation_loss=0.7242
epoch = 100 train_loss = 0.5208, validation_loss=0.5807
epoch = 150 train_loss = 0.4671, validation_loss=0.5397
epoch = 200 train_loss = 0.4408, validation_loss=0.5231
epoch = 250 train_loss = 0.4259, validation_loss=0.5155
epoch = 300 train_loss = 0.4168, validation_loss=0.5115
epoch = 350 train_loss = 0.411, validation_loss=0.5092
epoch = 400 train_loss = 0.4071, validation_loss=0.5079
epoch = 450 train_loss = 0.4045, validation_loss=0.507
epoch = 500 train_loss = 0.4027, validation_loss=0.5064
epoch = 550 train_loss = 0.4015, validation_loss=0.506
epoch = 600 train_loss = 0.4007, validation_loss=0.5058
epoch = 650 train_loss = 0.4001, validation_loss=0.5056
epoch = 700 train_loss = 0.3997, validation_loss=0.5055
epoch = 750 train_loss = 0.3994, validation_loss=0.5054
epoch = 800 train_loss = 0.3992, validation_loss=0.5053
epoch = 850 train_loss = 0.3991, validation_loss=0.505

In [319]:
test(model_computers, amazon_computers)

Train Accuracy = 0.8906467315716272
Valid Accuracy = 0.85625
Test  Accuracy = 0.86325


## Amazon Photo

In [325]:
model_photo =  SGC(number_features=745, number_classes=8, k=2)

In [327]:
train(model_photo, amazon_photo, lr=0.2)

epoch = 0 train_loss = 2.0716, validation_loss=2.0717
epoch = 50 train_loss = 0.4832, validation_loss=0.541
epoch = 100 train_loss = 0.3686, validation_loss=0.4379
epoch = 150 train_loss = 0.3288, validation_loss=0.4127
epoch = 200 train_loss = 0.3094, validation_loss=0.4026
epoch = 250 train_loss = 0.2986, validation_loss=0.3982
epoch = 300 train_loss = 0.2922, validation_loss=0.3963
epoch = 350 train_loss = 0.288, validation_loss=0.3956
epoch = 400 train_loss = 0.2853, validation_loss=0.3954
epoch = 450 train_loss = 0.2835, validation_loss=0.3954
epoch = 500 train_loss = 0.2822, validation_loss=0.3955
epoch = 550 train_loss = 0.2813, validation_loss=0.3956
epoch = 600 train_loss = 0.2807, validation_loss=0.3958
epoch = 650 train_loss = 0.2803, validation_loss=0.3959
epoch = 700 train_loss = 0.28, validation_loss=0.396
epoch = 750 train_loss = 0.2798, validation_loss=0.3961
epoch = 800 train_loss = 0.2796, validation_loss=0.3962
epoch = 850 train_loss = 0.2795, validation_loss=0.3963


In [328]:
test(model_photo, amazon_photo)

Train Accuracy = 0.9315068493150684
Valid Accuracy = 0.912
Test  Accuracy = 0.9075


# Results GCN

## Amazon Computers

In [360]:
model_computers_gcn =  GCN(in_channels=767, hidden_channels=64, out_channels=10)

In [361]:
train_gcn(model_computers_gcn, amazon_computers)

In [362]:
test_gcn(model_computers_gcn, amazon_computers)

Train Accuracy = 0.9431502086230876
Valid Accuracy = 0.911
Test  Accuracy = 0.90975


## Amazon Photo

In [355]:
model_photo_gcn =  GCN(in_channels=745, hidden_channels=64, out_channels=8)

In [356]:
train_gcn(model_photo_gcn, amazon_photo)

In [357]:
test_gcn(model_photo_gcn, amazon_photo)

Train Accuracy = 0.9605479452054795
Valid Accuracy = 0.9415
Test  Accuracy = 0.9405
